In [ ]:
import os
import cv2
import face_recognition
import numpy as np
import socketio
import http.client
import json
import tkinter as tk
from tkinter import ttk

# Global variable to control the running state of the main loop
running = True

# Function to download images from the server and save them locally
def download_images(username, projectname):
    conn = http.client.HTTPSConnection("graduation-api-zaj9.onrender.com")
    payload = ''
    headers = {}
    query = f"/api/v1/files/picture?username={username}&projectname={projectname}"
    conn.request("GET", query, payload, headers)
    res = conn.getresponse()
    data = res.read()
    image_data = json.loads(data.decode("utf-8"))

    # Ensure the directory to store the images exists
    os.makedirs("owners", exist_ok=True)

    # Extract and store the binary data of the images
    for img in image_data:
        img_id = img["_id"]
        project_id = img["projectId"]
        picture_name = img["pictureName"]

        img_type = img["contentType"].split('/')[-1]
        binary_data = bytes(img["data"]["data"])
        
        # Define the path to save the image
        img_dir = f"owners/{project_id}"
        img_path = f"{img_dir}/{picture_name}"

        # Ensure the directory exists
        os.makedirs(img_dir, exist_ok=True)
        
        # Write the binary data to a file
        with open(img_path, "wb") as img_file:
            img_file.write(binary_data)
        
        print(f"Image saved to {img_path}")

# Function to create the GUI
def create_gui():
    def on_submit():
        username = username_entry.get()
        projectname = projectname_entry.get()
        if username and projectname:
            root.destroy()
            main(username, projectname)
        else:
            error_label.config(text="Please enter both username and projectname.")

    root = tk.Tk()
    root.title("Download Images")

    # Set window size and center it on the screen
    window_width = 400
    window_height = 200

    screen_width = root.winfo_screenwidth()
    screen_height = root.winfo_screenheight()

    position_top = int(screen_height / 2 - window_height / 2)
    position_right = int(screen_width / 2 - window_width / 2)

    root.geometry(f"{window_width}x{window_height}+{position_right}+{position_top}")

    mainframe = ttk.Frame(root, padding="20 20 20 20")
    mainframe.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))

    ttk.Label(mainframe, text="Username:").grid(row=1, column=1, sticky=tk.W)
    username_entry = ttk.Entry(mainframe, width=25)
    username_entry.grid(row=1, column=2, sticky=(tk.W, tk.E))

    ttk.Label(mainframe, text="Projectname:").grid(row=2, column=1, sticky=tk.W)
    projectname_entry = ttk.Entry(mainframe, width=25)
    projectname_entry.grid(row=2, column=2, sticky=(tk.W, tk.E))

    submit_button = ttk.Button(mainframe, text="Submit", command=on_submit)
    submit_button.grid(row=3, column=2, sticky=tk.E)

    error_label = ttk.Label(mainframe, text="", foreground="red")
    error_label.grid(row=4, column=1, columnspan=2)

    for child in mainframe.winfo_children():
        child.grid_configure(padx=10, pady=10)

    root.mainloop()

# Create a Socket.IO client instance
sio = socketio.Client()

# Define event handlers
@sio.event
def connect():
    print("Successfully connected to the server")

@sio.event
def connect_error(data):
    print("Failed to connect to the server")
    print(data)

@sio.event
def disconnect():
    print("Disconnected from the server")

# Adjusted event handler for receiving messages
@sio.on('message1')
def on_message1(data):
    print("Received message1:", data)

# Adjusted event handler for test response
@sio.on('test response')
def on_test_response(data):
    print("Test response received:", data)

# Adjusted event handler for rulesResponse
@sio.on('rulesResponse')
def on_rules_response(data):
    print("Rules response received:", data)

# Function to send a message to the server
def send_room_message(message_value):
    sio.emit("cameraEvent", message_value)

# Connect to the Socket.IO server
server_url = "http://143.47.53.106:4934/"  # Ensure the correct protocol (https)
try:
    sio.connect(server_url, wait_timeout=20)  # Increase timeout
except socketio.exceptions.ConnectionError as e:
    print("Connection failed:", e)

# Function to encode faces from images in a given directory and its subdirectories
def encode_faces(directory):
    encoded_faces = {}
    for root, dirs, files in os.walk(directory):
        for filename in files:
            if filename.endswith(".jpg") or filename.endswith(".png"):
                img_path = os.path.join(root, filename)
                print(f"Processing image: {img_path}")
                img = face_recognition.load_image_file(img_path)
                encodings = face_recognition.face_encodings(img)
                if encodings:  # Check if any encodings are returned
                    encoding = encodings[0]
                    person_name = os.path.splitext(filename)[0]  # Image filename without extension
                    # Determine the project ID
                    if root == directory:
                        project_id = person_name
                    else:
                        project_id = os.path.basename(root)
                    encoded_faces[person_name] = (encoding, project_id)
                else:
                    print(f"No face found in image {img_path}")
    return encoded_faces

def main(username, projectname):
    global running
    # Download images from the server
    download_images(username, projectname)

    # Encode faces from 'owners' directory and its subdirectories
    encoded_faces = encode_faces("owners")

    # Initialize the camera
    cap = cv2.VideoCapture(0)

    # Keep track of the sent messages to avoid sending multiple times for the same face
    sent_messages = set()

    while running:
        ret, frame = cap.read()
        if not ret:
            break

        # Convert the frame from BGR to RGB
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Find all face locations and face encodings in the current frame
        face_locations = face_recognition.face_locations(rgb_frame)
        face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

        for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
            # Check if the face is a match for any known face
            matches = face_recognition.compare_faces([enc[0] for enc in encoded_faces.values()], face_encoding)
            name = "Unknown"
            project_id = ""

            if True in matches:
                first_match_index = matches.index(True)
                name = list(encoded_faces.keys())[first_match_index]
                project_id = encoded_faces[name][1]
                if name not in sent_messages:
                    # Send the person's name and project ID to the server
                    send_room_message({"name": name, "project_id": project_id})
                    sent_messages.add(name)

            # Draw a box around the face
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
            # Draw a label with the person's name below the face
            cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

        # Display the resulting image
        cv2.imshow('Video', frame)

        # Hit 'q' on the keyboard to quit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            running = False

    # Release handle to the webcam
    cap.release()
    cv2.destroyAllWindows()

# Run the GUI to get inputs and start the main function
if __name__ == "__main__":
    create_gui()
    # Wait for messages (this keeps the connection alive)
    #sio.wait()
